In [1]:
from ortools.linear_solver import pywraplp

### 1. Food manufacture 1

#### Entitities

- $V \in 1, \ldots, 5$: the types of oil. $v_1$ = VEG1, $v_2$ = VEG2, $v_5$ = OIL3.
- $M \in 1, \ldots, 6$: the six months in the problem.

#### Parameters

- $P_{vm} \, \, \forall v \in V, m \in M$: price to buy oil $v$ in month $m$.

- $H_v \, \, \forall v \in V$: hardness of each oil $v$.

#### Decision variables

- $x_{vm}$: amount of each oil to _buy_ in each month.

- $u_{vm}$: amount of each oil to _refine_ in each month.

- $n_{vm}$: amount of each oil that is in storage at the end of each month.

- $y_m$: amount of product we make each month.

#### Objective function

$$ \max 150 \sum_{m \in M} y_m - 5 \sum_{m \in M, v \in V} n_{vm} - \sum_{v \in V, m \in M} P_{vm}x_{vm}$$

#### Constraints

subject to:

1. $$n_{vm} = n_{vm-1} + x_{vm} - u_{vm} \, \, \forall v \in V, m \in 2,\ldots, 6$$

2. $$n_{v1} = 500 \, \, \forall v \in V$$

3. $$n_{v6} >= 500 \, \,  \forall v \in V$$

4. $$n_{vm} \leq 1000 \, \, \forall v \in V, m \in M$$

5. $$y_m = \sum_{v \in V} u_{vm} \, \, \forall m \in M$$

6. $$3y_m \leq \sum_{v \in V} H_v u_{vm} \, \, \forall m \in M$$

7. $$6y_m \geq \sum_{v \in V} H_v u_{vm} \, \, \forall m \in M $$ 

8. $$u_{1m} + u_{2m} \leq 200 \, \, \forall m \in M$$

9. $$u_{3m} + u_{4m} + u_{5m} \leq 250 \, \, \forall m \in M$$

In [2]:
M = range(1, 7) # months
V = range(1, 6) # types of oil

P = {1:{1:110, 2:130, 3:110, 4:120, 5:100, 6:90},
     2:{1:120, 2:130, 3:140, 4:110, 5:120, 6:100},
     3:{1:130, 2:110, 3:130, 4:120, 5:150, 6:140},
     4:{1:110, 2:90, 3:100, 4:120, 5:110, 6:80},
     5:{1:115, 2:115, 3:95, 4:125, 5:105, 6:135}} # purchase price

H = {1:8.8, 2:6.1, 3:2.0, 4:4.2, 5:5.0}

In [3]:
## Setup solver
solver = pywraplp.Solver("FoodManufacture1", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_VM = {v: {m: solver.NumVar(lb=0, ub=inf, name = f"x_{v}_{m}") 
            for m in M} for v in V}

u_VM = {v: {m: solver.NumVar(lb=0, ub=250, name = f"u_{v}_{m}") 
            for m in M} for v in V}

n_VM = {v: {m: solver.NumVar(lb=0, ub=1000, name = f"n_{v}_{m}") 
            for m in M} for v in V}

y_M = {m: solver.NumVar(lb=0, ub=inf, name = f"y_{m}") for m in M}

## Objective function
solver.Maximize((150 * solver.Sum([y_M[m] for m in M])) # sales
                - (5 * solver.Sum([n_VM[v][m] for v in V for m in M])) # storage
                - solver.Sum([P[v][m] * x_VM[v][m] for v in V for m in M])) # buying

## Constraints

# continuity constraints
for v in V:
    solver.Add(n_VM[v][1] == 500 + x_VM[v][1] - u_VM[v][1])
for m in M[1:]:
    for v in V:
        solver.Add(n_VM[v][m] == n_VM[v][m-1] + x_VM[v][m] - u_VM[v][m])
    
# must have enough units at the end
for v in V:
    solver.Add(n_VM[v][6] >= 500)
    
# storage capacity limits
for v in V:
    for m in M:
        solver.Add(n_VM[v][m] <= 1000)
        
# final amount of product
for m in M:
    solver.Add(y_M[m] == solver.Sum([u_VM[v][m] for v in V]))
               
# hardness limits
for m in M:
    solver.Add(3 * y_M[m] <= solver.Sum([H[v] * u_VM[v][m] for v in V]))
    solver.Add(6 * y_M[m] >= solver.Sum([H[v] * u_VM[v][m] for v in V]))

# capacity limits
for m in M:
    solver.Add(u_VM[1][m] + u_VM[2][m] <= 200)
    solver.Add(u_VM[3][m] + u_VM[4][m] + u_VM[5][m] <= 250)

In [4]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £107843 of profit.


### 2. Food manufacture 2

As before, but with the addition of a new variable , which takes value 1 if an oil is used, 0 otherwise. 

#### Extra decision variables

- $\delta_{vm} \in \{ 0, 1\}$

#### Extra constraints

10. $$\sum_{v \in V} \delta_{vm} \leq 3 \, \, \forall m \in M$$

11. $$\delta_{m5} \geq \delta_{m1} \, \, \forall m \in M$$

12. $$\delta_{m5} \geq \delta_{m2} \, \, \forall m \in M$$

13. $$u_{vm} \geq 20 \delta_{vm} \, \, \forall m \in M$$

14. $$u_{vm} \leq 250 \delta_{vm} \, \, \forall m \in M$$

In [5]:
M = range(1, 7) # months
V = range(1, 6) # types of oil

P = {1:{1:110, 2:130, 3:110, 4:120, 5:100, 6:90},
     2:{1:120, 2:130, 3:140, 4:110, 5:120, 6:100},
     3:{1:130, 2:110, 3:130, 4:120, 5:150, 6:140},
     4:{1:110, 2:90, 3:100, 4:120, 5:110, 6:80},
     5:{1:115, 2:115, 3:95, 4:125, 5:105, 6:135}} # purchase price

H = {1:8.8, 2:6.1, 3:2.0, 4:4.2, 5:5.0}

In [6]:
## Setup solver
solver = pywraplp.Solver("FoodManufacture2", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_VM = {v: {m: solver.NumVar(lb=0, ub=inf, name = f"x_{v}_{m}") 
            for m in M} for v in V}

u_VM = {v: {m: solver.NumVar(lb=0, ub=250, name = f"u_{v}_{m}") 
            for m in M} for v in V}

n_VM = {v: {m: solver.NumVar(lb=0, ub=1000, name = f"n_{v}_{m}") 
            for m in M} for v in V}

y_M = {m: solver.NumVar(lb=0, ub=inf, name = f"y_{m}") for m in M}

delta_VM = {v: {m: solver.IntVar(lb=0, ub=1, name = f"delta_{v}_{m}")
                for m in M} for v in V}

## Objective function
solver.Maximize((150 * solver.Sum([y_M[m] for m in M])) # sales
                - (5 * solver.Sum([n_VM[v][m] for v in V for m in M])) # storage
                - solver.Sum([P[v][m] * x_VM[v][m] for v in V for m in M])) # buying

## Constraints

# continuity constraints
for v in V:
    solver.Add(n_VM[v][1] == 500 + x_VM[v][1] - u_VM[v][1])
for m in M[1:]:
    for v in V:
        solver.Add(n_VM[v][m] == n_VM[v][m-1] + x_VM[v][m] - u_VM[v][m])
    
# must have enough units at the end
for v in V:
    solver.Add(n_VM[v][6] >= 500)
    
# storage capacity limits
for v in V:
    for m in M:
        solver.Add(n_VM[v][m] <= 1000)
        
# final amount of product
for m in M:
    solver.Add(y_M[m] == solver.Sum([u_VM[v][m] for v in V]))
               
# hardness limits
for m in M:
    solver.Add(3 * y_M[m] <= solver.Sum([H[v] * u_VM[v][m] for v in V]))
    solver.Add(6 * y_M[m] >= solver.Sum([H[v] * u_VM[v][m] for v in V]))

# capacity limits
for m in M:
    solver.Add(u_VM[1][m] + u_VM[2][m] <= 200)
    solver.Add(u_VM[3][m] + u_VM[4][m] + u_VM[5][m] <= 250)
    
# Cannot use more than 3 types of oil
for m in M:
    solver.Add(solver.Sum([delta_VM[v][m] for v in V]) <= 3)

# If we use v1 or v2, we must use o3
for m in M:
    solver.Add(delta_VM[5][m] >= delta_VM[1][m])
    solver.Add(delta_VM[5][m] >= delta_VM[2][m])
    
# If we use an oil, we must use 20+ tons
for m in M:
    for v in V:
        solver.Add(u_VM[v][m] >= 20 * delta_VM[v][m])
        
# If delta=zero, used=0
for m in M:
    for v in V:
        solver.Add(u_VM[v][m] <= 251 * delta_VM[v][m])

In [7]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £100279 of profit.


### 3. Factory planning 1


#### Entities

- $p \in P$: different products we can manufacture.
- $m \in M$: months over which we can manufacture products.
- $f \in F$: different factory machines involved in the production process.

#### Parameters

- $c_{fm}\, \, \forall f \in F, m \in M$: number of machines $f$ available in the month $m$.
- $t_{pf} \, \, \forall p \in P, f \in F$: amount of time it takes to make one unit of product $p$ on machine $f$.
- $h$: number of hours each machine is available for in each month (384).
- $n_{p0} = 0 \, \, \forall p \in P$: total stock we have in at the start of the process to sell.
- $d_{pm} \, \, \forall p \in P, m \in M$: market capacity for each product and month.
- $v_{p} \, \, \forall p \in P$: profit we make for each unit of product $p$.

#### Decision variables

- $x_{pm} \, \, \forall p \in P, m \in M$: amount of each product to make in each month.
- $s_{pm} \, \, \forall p \in P, m \in M$: amount of each product to sell each month.
- $n_{pm} \, \, \forall p \in P, m \in M$: amount of each product left in storage in each month.

#### Objective function

$$ \max \sum_{p \in P, m \in M}{v_p \cdot s_{pm}} - 0.5\sum_{p \in P, m \in M}{n_{pm}}$$

#### Constraints

1. $$n_{pm} \leq 100 \, \, \forall p \in P, m \in M$$

2. $$n_{p6} \geq 50 \, \, \forall p \in P$$

3. $$\sum_{p \in P} t_{pf} \cdot x_{pm} \leq h \cdot c_{fm} \, \, \forall f \in F, m \in M$$

4. $$s_{pm} \leq d_{pm} \, \, \forall p \in P, m \in M$$

5. $$s_{pm} \leq x_{pm} + n_{pm-1} \, \, \forall p \in P, m \in M$$

6. $$n_{pm} = x_{pm} + n_{pm-1} - s_{pm} \, \, \forall p \in P, m \in M$$

In [8]:
## Entities
M = range(1, 7) # months
P = range(1, 8) # types of product
F = range(1, 6) # types of machine process

## Parameters
c_FM = {1: {1: 3, 2: 4, 3: 4, 4: 4, 5: 3, 6: 4},
        2: {1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 2},
        3: {1: 3, 2: 1, 3: 3, 4: 3, 5: 3, 6: 2},
        4: {1: 1, 2: 1, 3: 0, 4: 1, 5: 1, 6: 1},
        5: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0}}

t_PF = {1: {1: 0.5, 2: 0.1, 3: 0.2, 4: 0.05, 5: 0.00},
        2: {1: 0.7, 2: 0.2, 3: 0.0, 4: 0.03, 5: 0.00},
        3: {1: 0.0, 2: 0.0, 3: 0.8, 4: 0.00, 5: 0.01},
        4: {1: 0.0, 2: 0.3, 3: 0.0, 4: 0.07, 5: 0.00},
        5: {1: 0.3, 2: 0.0, 3: 0.0, 4: 0.10, 5: 0.05},
        6: {1: 0.2, 2: 0.6, 3: 0.0, 4: 0.00, 5: 0.00},
        7: {1: 0.5, 2: 0.0, 3: 0.6, 4: 0.08, 5: 0.05}}

d_PM = {1: {1: 500, 2: 600, 3: 300, 4: 200, 5: 0, 6: 500},
        2: {1: 1000, 2: 500, 3: 600, 4: 300, 5: 100, 6: 500},
        3: {1: 300, 2: 200, 3: 0, 4: 400, 5: 500, 6: 100},
        4: {1: 300, 2: 0, 3: 0, 4: 500, 5: 100, 6: 300},
        5: {1: 800, 2: 400, 3: 500, 4: 200, 5: 1000, 6: 1100},
        6: {1: 200, 2: 300, 3: 400, 4: 0, 5: 300, 6: 500},
        7: {1: 100, 2: 150, 3: 100, 4: 100, 5: 0, 6: 60}}

v_P = {1: 10, 2: 6, 3: 8, 4: 4, 5: 11, 6: 9, 7: 3}

h = 384

In [9]:
## Setup solver
solver = pywraplp.Solver("FactoryPlanning1", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"x_{p}_{m}")
            for m in M} for p in P}

s_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"s_{p}_{m}")
            for m in M} for p in P}

n_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"n_{p}_{m}")
            for m in M} for p in P}

## Objective function
solver.Maximize(solver.Sum([v_P[p] * s_PM[p][m] for p in P for m in M]) # sales profit
                - 0.5 * solver.Sum([n_PM[p][m] for p in P for m in M])) # storage costs

## Constraints

# storage capacity constraint
for p in P:
    for m in M:
        solver.Add(n_PM[p][m] <= 100)

# must have 50+ of each product at the end of the process
m = 6
for p in P:
    solver.Add(n_PM[p][m] >= 50)
    
# hourly limits on each machine per month
for f in F:
    for m in M:
        solver.Add(solver.Sum([t_PF[p][f] * x_PM[p][m] for p in P]) 
                   <= h * c_FM[f][m])
        
# cannot sell more than the demand for that month
for p in P:
    for m in M:
        solver.Add(s_PM[p][m] <= d_PM[p][m])
        
# cannot sell more than we have in stock for the month
for p in P:
    solver.Add(s_PM[p][1] <= x_PM[p][1] + 0)
    for m in M[1:]:
        solver.Add(s_PM[p][m] <= x_PM[p][m] + n_PM[p][m-1])
        
# continuity equation for the stock left at the end of each month
for p in P:
    n_PM[p][m] == x_PM[p][1] + 0 - s_PM[p][1]
    for m in M[1:]:
        solver.Add(n_PM[p][m] == x_PM[p][m] + n_PM[p][m-1] - s_PM[p][m])

In [10]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £93715 of profit.


### 4. Factory planning 2


#### Entities

- $p \in P$: different products we can manufacture.
- $m \in M$: months over which we can manufacture products.
- $f \in F$: different factory machines involved in the production process.

#### Parameters

- $t_{pf} \, \, \forall p \in P, f \in F$: amount of time it takes to make one unit of product $p$ on machine $f$.
- $h$: number of hours each machine is available for in each month (384).
- $n_{p0} = 0 \, \, \forall p \in P$: total stock we have in at the start of the process to sell.
- $d_{pm} \, \, \forall p \in P, m \in M$: market capacity for each product and month.
- $v_{p} \, \, \forall p \in P$: profit we make for each unit of product $p$.
- $C_{f} \, \, \forall f \in F$: total number of each type of machine.
- $Z_{f} \, \, \forall f \in F$: total amount of maintenance of each machine that must be performed in the 6 month period.

#### Decision variables

- $x_{pm} \, \, \forall p \in P, m \in M$: amount of each product to make in each month.
- $s_{pm} \, \, \forall p \in P, m \in M$: amount of each product to sell each month.
- $n_{pm} \, \, \forall p \in P, m \in M$: amount of each product left in storage in each month.
- $z_{fm} \, \, \forall f \in F, m \in M$: total number of each type of machine that are down for maintenance in each month.
- $c_{fm}\, \, \forall f \in F, m \in M$: number of machines $f$ available in the month $m$.

#### Objective function

$$ \max \sum_{p \in P, m \in M}{v_p \cdot s_{pm}} - 0.5\sum_{p \in P, m \in M}{n_{pm}}$$

#### Constraints

1. $$n_{pm} \leq 100 \, \, \forall p \in P, m \in M$$

2. $$n_{p6} \geq 50 \, \, \forall p \in P$$

3. $$\sum_{p \in P} t_{pf} \cdot x_{pm} \leq h \cdot c_{fm} \, \, \forall f \in F, m \in M$$

4. $$s_{pm} \leq d_{pm} \, \, \forall p \in P, m \in M$$

5. $$s_{pm} \leq x_{pm} + n_{pm-1} \, \, \forall p \in P, m \in M$$

6. $$n_{pm} = x_{pm} + n_{pm-1} - s_{pm} \, \, \forall p \in P, m \in M$$

7. $$c_{fm} \leq C_{f} - z_{fm} \, \, \forall f \in F, m \in M$$

8. $$\sum_{m \in M} z_{fm} \geq Z_{f} \, \, \forall f \in F$$

9. $$z_{fm} \geq 0 \, \, \forall f \in F, m \in M$$

In [11]:
## Entities
M = range(1, 7) # months
P = range(1, 8) # types of product
F = range(1, 6) # types of machine process

## Parameters

t_PF = {1: {1: 0.5, 2: 0.1, 3: 0.2, 4: 0.05, 5: 0.00},
        2: {1: 0.7, 2: 0.2, 3: 0.0, 4: 0.03, 5: 0.00},
        3: {1: 0.0, 2: 0.0, 3: 0.8, 4: 0.00, 5: 0.01},
        4: {1: 0.0, 2: 0.3, 3: 0.0, 4: 0.07, 5: 0.00},
        5: {1: 0.3, 2: 0.0, 3: 0.0, 4: 0.10, 5: 0.05},
        6: {1: 0.2, 2: 0.6, 3: 0.0, 4: 0.00, 5: 0.00},
        7: {1: 0.5, 2: 0.0, 3: 0.6, 4: 0.08, 5: 0.05}}

d_PM = {1: {1: 500, 2: 600, 3: 300, 4: 200, 5: 0, 6: 500},
        2: {1: 1000, 2: 500, 3: 600, 4: 300, 5: 100, 6: 500},
        3: {1: 300, 2: 200, 3: 0, 4: 400, 5: 500, 6: 100},
        4: {1: 300, 2: 0, 3: 0, 4: 500, 5: 100, 6: 300},
        5: {1: 800, 2: 400, 3: 500, 4: 200, 5: 1000, 6: 1100},
        6: {1: 200, 2: 300, 3: 400, 4: 0, 5: 300, 6: 500},
        7: {1: 100, 2: 150, 3: 100, 4: 100, 5: 0, 6: 60}}

v_P = {1: 10, 2: 6, 3: 8, 4: 4, 5: 11, 6: 9, 7: 3}

C_F = {1: 4, 2: 2, 3: 3, 4: 1, 5: 1}

Z_F = {1: 2, 2: 2, 3: 3, 4: 1, 5: 1}

h = 384

In [12]:
## Setup solver
solver = pywraplp.Solver("FactoryPlanning2", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"x_{p}_{m}")
            for m in M} for p in P}

s_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"s_{p}_{m}")
            for m in M} for p in P}

n_PM = {p: {m: solver.NumVar(lb=0, ub=inf, name=f"n_{p}_{m}")
            for m in M} for p in P}

z_FM = {f: {m: solver.IntVar(lb=0, ub=Z_F[f], name=f"z_{f}_{m}")
            for m in M} for f in F}

c_FM = {f: {m: solver.IntVar(lb=0, ub=C_F[f], name=f"c_{f}_{m}")
            for m in M} for f in F}

## Objective function
solver.Maximize(solver.Sum([v_P[p] * s_PM[p][m] for p in P for m in M]) # sales profit
                - 0.5 * solver.Sum([n_PM[p][m] for p in P for m in M])) # storage costs

## Constraints

# storage capacity constraint
for p in P:
    for m in M:
        solver.Add(n_PM[p][m] <= 100)

# must have 50+ of each product at the end of the process
m = 6
for p in P:
    solver.Add(n_PM[p][m] >= 50)
    
# hourly limits on each machine per month
for f in F:
    for m in M:
        solver.Add(solver.Sum([t_PF[p][f] * x_PM[p][m] for p in P]) 
                   <= h * c_FM[f][m])
        
# cannot sell more than the demand for that month
for p in P:
    for m in M:
        solver.Add(s_PM[p][m] <= d_PM[p][m])
        
# cannot sell more than we have in stock for the month
for p in P:
    solver.Add(s_PM[p][1] <= x_PM[p][1] + 0)
    for m in M[1:]:
        solver.Add(s_PM[p][m] <= x_PM[p][m] + n_PM[p][m-1])
        
# continuity equation for the stock left at the end of each month
for p in P:
    n_PM[p][m] == x_PM[p][1] + 0 - s_PM[p][1]
    for m in M[1:]:
        solver.Add(n_PM[p][m] == x_PM[p][m] + n_PM[p][m-1] - s_PM[p][m])
        
# capacity of each machine in each month
for f in F:
    for m in M:
        solver.Add(c_FM[f][m] <= C_F[f] - z_FM[f][m])

# make sure each machine gets the required amount of maintenance
for f in F:
    solver.Add(solver.Sum([z_FM[f][m] for m in M]) >= Z_F[f])

In [13]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £108855 of profit.


### 5. Manpower planning

#### Entities

- $s \in S$: skill levels of workers (1 = unskilled, 2 = semi-skilled, 3 = skilled).
- $y \in Y$: year of operation (1, 2, 3).

#### Parameters

- $N_{sy} \, \, \forall s \in S, y \in Y$: number of workers of each type required in each year.
- $n_{s0} \, \, \forall s \in S$: number of workers at the start of the process.
- $C_{us} \, \, \forall s \in \{1, 2\}$: cost to upskill a worker up one level.
- $C_{rs} \, \, \forall s \in S$: cost to make a worker redundant.
- $C_{os} \, \, \forall s \in S$: cost to have extra workers in the company.
- $C_{ps} \, \, \forall s \in S$: cost incurred for each part-time worker. 
- $N_{hs} \forall s \in S$: maximum number of workers we can hire each year.
- $N_{u1} = 200$: maximum number of people we are allowed to upskill from unskilled to semi-skilled.
- $f_{u23} = 0.25$: fraction of the skilled workforce we are allowed to upskill from the semi-skilled workforce.
- $N_{o} = 150$: total number of workers we are allowed to overstaff by. 
- $N_{p} = 50$: maximum allowed number of workers on part-time per skill level.
- $f_{s} \, \, \forall s \in S$: fraction of long-term (1+ years service) workforce retained year on year.
- $g_{s} \, \, \forall s \in S$: fraction of newly-hired (1 years service) workforce retained year on year.

#### Decision variables

- $n_{sy}  \geq 0 \, \, \forall s \in S, y \in Y$: total workforce size by skill level each year. 
- $h_{sy} \geq 0 \, \, \forall s \in S, y \in Y$: total number of people to hire by skill level each year.
- $u_{sy} \geq 0 \, \, \forall s \in \{1, 2\}, y \in Y$: total number of workers to upskill each year.
- $d_{sy} \geq 0 \, \, \forall s \in \{2, 3\}, y \in Y$: total number of workers to downskill each year. 
- $r_{sy} \geq 0 \, \, \forall s \in S, y \in Y$: total number of workers to make redundant each year.
- $p_{sy} \geq 0 \, \, \forall s \in S, y \in Y$: total number of workers to put on part-time work each year. 
- $o_{sy} \geq 0 \, \, \forall s \in S, y \in Y$: total number of workers to overman by each year. 

#### Objective

1. To minimise redundancy: 

$$ \min(\sum_{s \in S, y \in Y} r_{sy})$$

2. To minimise costs:

$$ \min(\sum_{s \in S, y \in Y} C_{rs} \cdot r_{sy} + C_{ps} \cdot p_{sy} + C_{os} \cdot o_{sy} + \sum_{s \in \{1, 2\}, y \in Y} C_{us} \cdot u_{sy})$$


#### Constraints

1. $\sum(n_{sy} \forall s \in S) \leq \sum(N_{sy} \forall s \in S) + N_{o} \forall \, \, y \in Y$
2. $n_{sy} - 0.5 \cdot p_{sy} \geq N_{sy} \, \, \forall \, \, s \in S, y \in Y$
3. $p_{sy} \leq N_{p} \, \, \forall \, \, s \in S, y \in Y$
4. $h_{sy} \leq N_{hs} \, \, \forall \, \, s \in S, y \in Y$
5. $u_{1y} \leq N_{u1} \, \, \forall \, \, y \in Y$
6. $u_{2y} \leq f_{u23} \cdot n_{3y} \, \, \forall \, \, y \in Y$
7. $n_{1y} = (1-f_{1})n_{1y-1} + (1-g_{1y})h_{1y} - r_{1y} + 0.5 \cdot d_{21} - u_{12} \, \, \forall \, \, y \in Y$
8. $n_{2y} = (1-f_{2})n_{2y-1} + (1-g_{2y})h_{2y} - r_{2y} + 0.5 \cdot d_{32} - u_{23} + (1-f_2)u_{12} - d_{21} \, \, \forall \, \, y \in Y$
9. $n_{3y} = (1-f_{3})n_{3y-1} + (1-g_{3y})h_{3y} - r_{3y} - d_{32} + (1-f_3)u_{23} \, \, \forall \, \, y \in Y$

In [14]:
## Entities
S = [1, 2, 3] # skill levels
Y = [1, 2, 3] # year

## Parameters
N_SY = {1: {1: 1000, 2: 1400, 3: 1000},
        2: {1: 500, 2: 2000, 3: 1500},
        3: {1: 0, 2: 2500, 3: 2000}}

n_S0 = {1: 2000, 2: 1500, 3: 1000}

C_uS = {1: 400, 2: 500}
C_rS = {1: 200, 2: 500, 3: 500}
C_oS = {1: 1500, 2: 2000, 3: 3000}
C_pS = {1: 500, 2: 400, 3: 400}

N_hS = {1: 500, 2: 800, 3: 500}
N_u1 = 200
f_u23 = .25
N_o = 150
N_p = 50

f_S = {1: .10, 2: .05, 3:.05}
g_S = {1: .25, 2: .20, 3: .1}

In [15]:
## Setup solver
solver = pywraplp.Solver("ManPowerPlanning", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
n_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"n_{s}_{y}")
        for s in S} for y in Y}
n_SY[0] = n_S0 # start value

h_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"h_{s}_{y}")
        for s in S} for y in Y}
h_SY[0] = {1: 0, 2: 0, 3: 0} # start value

u_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"u_{s}_{y}")
        for s in S[:-1]} for y in Y}

d_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"d_{s}_{y}")
        for s in S[1:]} for y in Y}

r_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"r_{s}_{y}")
        for s in S} for y in Y}

p_SY = {y: {s: solver.IntVar(lb=0, ub=inf, name=f"p_{s}_{y}")
        for s in S} for y in Y}

## Objective function
solver.Minimize(solver.Sum([r_SY[y][s] for s in S for y in Y])) # minimise redundancies

## Constraints
for s in S:
    for y in Y:
        solver.Add(n_SY[y][s] - 0.5 * p_SY[y][s] >= N_SY[y][s])
        solver.Add(p_SY[y][s] <= N_p)
        solver.Add(h_SY[y][s] <= N_hS[s])
        
for y in Y:
    solver.Add(u_SY[y][1] <= N_u1)
    solver.Add(u_SY[y][2] <= f_u23 * n_SY[y][3])
    solver.Add(solver.Sum([n_SY[y][s] for s in S]) <= solver.Sum(N_SY[y][s] for s in S) + N_o)
    
    solver.Add(n_SY[y][1] == (((1-f_S[1]) * n_SY[y-1][1]) # old worker attrition
                              + ((1-g_S[1]) * h_SY[y][1]) # new hires + attrition
                              - (r_SY[y][1]) # redundancies
                              + (0.5 * d_SY[y][2]) # downskills
                              - (u_SY[y][1]) # upskills
                              )
              )
            
    solver.Add(n_SY[y][2] == (((1-f_S[2]) * n_SY[y-1][2]) # old worker attrition
                              + ((1-g_S[2]) * h_SY[y][2]) # new hires + attrition
                              - (r_SY[y][2]) # redundancies
                              + (0.5 * d_SY[y][3]) # downskills
                              + ((1-f_S[2]) * u_SY[y][1]) # upskills
                              - (d_SY[y][2]) # downskills
                              - (u_SY[y][2]) # upskills
                              )
              )
            
    solver.Add(n_SY[y][3] == (((1-f_S[3]) * n_SY[y-1][3]) # old worker attrition
                              + ((1-g_S[3]) * h_SY[y][3]) # new hires + attrition
                              - (r_SY[y][3]) # redundancies
                              - (d_SY[y][3]) # downskills
                              + ((1-f_S[3]) * u_SY[y][2])) # upskills
              )
    
        

In [16]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy requires {int(round(obj, 0))} redundancies.")

Solution found. Optimal policy requires 877 redundancies.
